In [1]:
import networkx as nx   
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler

In [2]:
G = nx.karate_club_graph()
labels = np.asarray([G.nodes[i]['club']!='Mr.Hi' for i in G.nodes]).astype(np.int64)

In [3]:
adj = nx.to_scipy_sparse_matrix(G).tocoo()
row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long)
col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long)
edge_index = torch.stack([row,col], dim = 0)


/var/folders/54/0ppv1xbn2h99kqvlhvh3sd3r0000gn/T/ipykernel_1678/3187827270.py:1: DeprecationWarning: 

The scipy.sparse array containers will be used instead of matrices
in Networkx 3.0. Use `to_scipy_sparse_array` instead.
  adj = nx.to_scipy_sparse_matrix(G).tocoo()


In [4]:
embeddings = np.array(list(dict(G.degree()).values()))
scale = StandardScaler()
embeddings = scale.fit_transform(embeddings.reshape(-1,1))

In [5]:
import torch
import pandas as pd
from torch_geometric.data import InMemoryDataset, Data
from sklearn.model_selection import train_test_split
import torch_geometric.transforms as T

In [18]:
class KarateDataset(InMemoryDataset):
    def __init__(self, transform= None):
        super(KarateDataset,self).__init__('.',transform,None,None)
        data = Data(edge_index= edge_index)
        data.num_nodes = G.number_of_nodes()
        data.x = torch.from_numpy(embeddings).type(torch.float32)
        y = torch.from_numpy(labels).type(torch.long)
        data.y = y.clone().detach()
        data.num_class = 2

        X_train,X_test, y_train, y_test = train_test_split(pd.Series(list(G.nodes())),
                                                           pd.Series(labels),
                                                           test_size = 0.30,
                                                           random_state=42)
        n_nodes = G.number_of_nodes()
        train_mask = torch.zeros(n_nodes,dtype=torch.bool)
        test_mask = torch.zeros(n_nodes,dtype=torch.bool)
        train_mask[X_train.index]=True
        test_mask[X_test.index] = True
        data['train_mask'] = train_mask
        data['test_mask'] = test_mask

        self.data , self.slices = self.collate([data])
    def _download(self):
        return

    def _process(self):
        return

    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)
    





In [19]:
dataset = KarateDataset()
data = dataset[0]
data

Data(edge_index=[2, 156], num_nodes=34, x=[34, 1], y=[34], num_class=2, train_mask=[34], test_mask=[34])

In [20]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [21]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(data.num_features,16)
        self.conv2 = GCNConv(16,int(data.num_class))
    def forward(self):
        x = data.x
        edge_index = data.edge_index
        x = F.relu(self.conv1(x,edge_index))
        x = F.dropout(x, training= self.training)
        x = self.conv2(x,edge_index)
        return F.log_softmax(x, dim = 1)

    

In [22]:
model = Net()

In [27]:
torch.manual_seed(42)
optimizer_name = 'Adam'
lr = 1
optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr = lr)
epochs = 20

def train():
    model.train()
    optimizer.zero_grad()
    predicts  = model()[data.train_mask]
    labels = data.y[data.train_mask]
    loss = F.nll_loss(predicts, labels)
    loss.backward()
    optimizer.step()

@torch.no_grad()
def test():
    model.eval()
    logits = model()
    mask1 = data['train_mask']
    pred1 = logits[mask1].max(1)[1]
    acc1 = pred1.eq(data.y[mask1]).sum().item()/mask1.sum().item()
    mask = data['test_mask']
    pred = logits[mask].max(1)[1]
    acc = pred.eq(data.y[mask]).sum().item()/mask.sum().item()
    return acc1, acc


In [28]:
for epoch in range(1,epochs):
    train()
    

In [29]:
train_acc, test_acc = test()
print(f'Train accuracy: {train_acc :.2f}')
print(f'Test accuracy: {test_acc :.2f}')


Train accuracy: 1.00
Test accuracy: 1.00
